# Chapter 9. A Pythonic Object

# Object Representations

# Vector Class Redux

In [65]:
from array import array
import math

class Vector2d:
#     __slots__ = ('__x', '__y')
    
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        # self.__x is fixed by a value passed at __init__
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(array(self.typecode, self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [5]:
v1= Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [7]:
x, y = v1
x, y

(3.0, 4.0)

In [8]:
v1

Vector2d(3.0, 4.0)

In [9]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [10]:
print(v1)

(3.0, 4.0)


In [11]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [12]:
abs(v1)

5.0

In [13]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

# classmethod Versus staticmethod
- staticmethod is not used at all because methods to need interact with class is implemented as classmethod, and methods not to touch class is defined in the module, where is out of class.

In [15]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [16]:
Demo.klassmeth()

(__main__.Demo,)

In [17]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [18]:
Demo.statmeth()

()

In [19]:
Demo.statmeth('spam')

('spam',)

# Formatted Displays

In [20]:
br1 = 1 / 2.43

In [21]:
format(br1, '0.4f')

'0.4115'

In [22]:
'1 BRL = {rate:0.2f} USD'.format(rate=br1)

'1 BRL = 0.41 USD'

In [23]:
format(42, 'b')

'101010'

In [24]:
format(2/3, '.1%')

'66.7%'

In [26]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'14:08:29'

In [27]:
"It's now {:%I:%M %p}".format(now)

"It's now 02:08 PM"

In [40]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [41]:
format(v1, '.3f')

'(3.000, 4.000)'

In [42]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [44]:
format(Vector2d(1, 1), 'p')

'(1.4142135623730951, 0.7853981633974483)'

In [48]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [49]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

# A Hashable Vector2d

In [50]:
v1 = Vector2d(3, 4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [51]:
set([v1])

TypeError: unhashable type: 'Vector2d'

In [54]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [55]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

# Private and "Protected" Attributes in Python

In [66]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [67]:
v1._Vector2d__x

3.0

In [68]:
v1.x

3.0

In [69]:
v1.x = 5.0

AttributeError: can't set attribute

In [70]:
v1.__x = 5.0

In [71]:
v1.x

3.0

# Saving Space with the `__slots__` Class Attribute
- Python stores instance attributes in per-instance dict named `__dict__` by default.
    - Dictionaries have a significant memory overhead because of the underlying hash table used to provide fast access.      
- `__slots__`
    - store the instance attributes in tuple instead of dict
    - Python only takes into account __slots__ defined in each class individually.  
      - should implement `__slots__` also when inherit class because superclass's `__slots__` can not be used.
    - defining `__slots__` means to tell the interpreter "There are all the instance attributes in this class"
    - `__slots__` should be used for optimization, not for programmer restraint (e.g. adding new attributes)

# Overriding Class Attributes

In [72]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [73]:
len(dumpd)

17

In [74]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [75]:
len(dumpf)

9

In [76]:
Vector2d.typecode

'd'

In [77]:
# used instead of `Vector2d.typecode = 'f'`
class ShortVector2d(Vector2d):
    typecode = 'f'